In [5]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import numpy_financial as npf

#Defining a function for calculate to duration
def calculate_duration(coupon_rate, maturity, interest_rate, coupon_freq, bps_scenario):
    periods = maturity * coupon_freq + 1 #How many coupon payment periods that we have?
    cash_flows = np.zeros(periods) # Initiliazie the array of cashflows
    cash_flows[0] = npf.pv(rate=interest_rate/coupon_freq, nper=periods-1, pmt=coupon_rate*100/coupon_freq, fv=100) #Calculating present value of bond
    cash_flows[1:-1] = coupon_rate/coupon_freq*100 
    cash_flow_per_period = cash_flows[1]
    cash_flows[-1] = cash_flow_per_period + 100 #Add the face value of the bond to the last cash flow
    #Prepare dataframe for our model
    df = pd.DataFrame({
        'Period': range(periods),
        'Cash_Flow': cash_flows,
        'Present_Value': 0,
        'Duration': 0,
        "Convexity": 0
    })
    #Loop over for calculations
    for i in range(1, periods):
        df.loc[i, 'Present_Value'] = df.loc[i, 'Cash_Flow'] / (1 + interest_rate/coupon_freq)**i
    for i in range(1, periods):
        df.loc[i, 'Duration'] = i * df.loc[i, 'Present_Value']
    for i in range(1,periods):
        df.loc[i, 'Convexity'] = 1/(1+interest_rate/coupon_freq)**2*df.loc[i, 'Present_Value']*(df.loc[i, 'Period']**2+df.loc[i, 'Period'])
        
    #Calculating results with formulas
    macaulay_duration = df['Duration'].sum() / df['Present_Value'].sum() / coupon_freq
    modified_duration = macaulay_duration / (1 + interest_rate/coupon_freq)
    duration_effect = -modified_duration * bps_scenario / 10000 * df['Present_Value'].sum()
    convexity = df['Convexity'].sum() / df['Present_Value'].sum() / coupon_freq**2
    convexity_effect = 0.5 * convexity * (bps_scenario/10000)**2 * df['Present_Value'].sum()
    duration_convexity_effect = duration_effect + convexity_effect
    new_bond_price = df['Present_Value'].sum() + duration_convexity_effect
    print(f"Present value of your bond is {df['Present_Value'].sum():.2f}")
    print(f"Macaulay duration of your bond is {macaulay_duration:.2f}")
    print(f"Modified duration of your bond is {modified_duration:.2f}")
    print(f"Modified duration effect of {bps_scenario} bps increase of the bond yield is {duration_effect:.2f}")
    print(f"Convexity effect of {bps_scenario} bps increase of the bond yield is {convexity_effect:.2f}")
    print(f"Total convexity and duration effect of {bps_scenario} bps increase of the bond yield is {duration_convexity_effect:.2f} and new present value of the bond price is {new_bond_price:.2f}")
    return df

calculate_duration(coupon_rate=0.06, maturity=10, interest_rate=0.10, coupon_freq=2, bps_scenario=100)

Present value of your bond is 75.08
Macaulay duration of your bond is 7.24
Modified duration of your bond is 6.89
Modified duration effect of 100 bps increase of the bond yield is -5.17
Convexity effect of 100 bps increase of the bond yield is 0.23
Total convexity and duration effect of 100 bps increase of the bond yield is -4.94 and new present value of the bond price is 70.13


,Period,Cash_Flow,Present_Value,Duration,Convexity
0,0,-75.075579,0.000000,0.000000,0.000000
1,1,3.000000,2.857143,2.857143,5.183026
2,2,3.000000,2.721088,5.442177,14.808645
3,3,3.000000,2.591513,7.774538,28.206942
4,4,3.000000,2.468107,9.872430,44.772924
5,5,3.000000,2.350578,11.752892,63.961320
6,6,3.000000,2.238646,13.431877,85.281760
7,7,3.000000,2.132044,14.924308,108.294298
8,8,3.000000,2.030518,16.244145,132.605263
9,9,3.000000,1.933827,17.404441,157.863408
